In [2]:
library(data.table)
library(IRanges)

In [3]:
library(parallel)
options(mc.cores = detectCores())

In [4]:
real <- c('A*02:01', 'A*03:01', 'B*13:02', 'B*51:01', 'C*01:02', 'C*06:02', 'DRB1*07:01', 'DRB1*11:01', 'DQB1*02:AB', 'DQB1*03:01', 'DPB1*02:01', 'DPB1*02:01')

In [5]:
dt <- fread('187521910.m8')

Read 5846112 rows and 12 (of 12) columns from 0.421 GB file in 00:00:07


In [6]:
setnames(dt, c('q', 't', 'iden', 'len', 'mis', 'gap', 'qf', 'qt', 'tf', 'tt', 'e', 'score'))
#dt <- dt[iden == 100]
tt <- dt[, gene := sub('\\*.+', '', t)]
setkey(dt, q)

In [7]:
nexpr <- dt[grepl('\\D-', t)]
dt <- dt[grepl('\\d-', t)]
tt <- dt[, c('LEN', 'GOOD', 'GOOD.N', 'BAD.N', 'N') := .(
    max(len), 
    paste(unique(gene[len == max(len)]), collapse = ','),
    sum(len == max(len) & iden == 100),
    sum(len < max(len) | iden < 100),
    .N)
, by = q]

In [8]:
good <- dt[len == LEN & iden == 100]
tt <- good[, pure := gene == GOOD]
dt <- dt[q %in% good$q]
bad <- dt[len < LEN | iden < 100]

In [9]:
ir.score <- function(tf, tt){
    ir <- reduce(IRanges(tf, tt), min.gapwidth = 0)
    return(list(
        nreads = length(tf),
        cov = sum(width(ir)),
        nseg = length(ir),
        nbreaks = sum(width(gaps(ir - 1)) == 2)
    ))
}

In [10]:
ir1 <- good[, ir.score(tf, tt), keyby = t]
ir2 <- good[pure==T, ir.score(tf, tt), keyby = t]
setnames(ir2, c('t', 'nreads2', 'cov2', 'nseg2', 'nbreaks2'))
score <- ir2[ir1][order(-cov)]

In [11]:
get.bad <- function(g){
    bad.score <- bad[gene == g & grepl(sprintf('^%s', g), GOOD), ir.score(tf, tt), keyby = t]
    setnames(bad.score, c('t', 'bad.nreads', 'bad.cov', 'bad.nseg', 'bad.nbreaks'))
    bad.score <- bad.score[score[grepl(sprintf("^%s", g), t)]][order(-cov)]
    return(bad.score)
}
bad.reads <- function(bad, hla){
    dt[q %in% bad[t == hla, q]]
}

In [12]:
head(get.bad('A')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,A*03:01-365,50,286,3,0,34,153,1,0,147,296,3,0
2,A*03:89-365,50,279,4,0,30,144,1,0,148,296,3,0
3,A*03:26-365,51,286,3,0,34,153,1,0,146,294,3,0
4,A*02:01-365,91,254,5,0,54,247,3,0,107,297,2,0
5,A*02:24-365,104,281,5,0,44,247,3,0,97,297,2,0
6,A*02:249-365,105,261,4,0,45,223,3,0,93,296,3,0


In [13]:
head(get.bad('B')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,B*51:01-362,54,237,2,0,17,151,3,0,80,297,2,0
2,B*51:193-362,54,237,2,0,17,151,3,0,80,297,2,0
3,B*51:96-337,53,227,2,0,16,129,2,0,79,295,2,0
4,B*51:187-362,55,237,2,0,17,151,3,0,78,294,3,0
5,B*51:04-362,47,237,2,0,8,99,3,0,71,291,3,0
6,B*51:192-362,66,237,2,0,9,119,3,0,65,292,3,0


In [14]:
head(get.bad('C')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,C*06:107-349,56,245,4,0,4,29,1,0,91,263,4,1
2,C*06:73-273,46,245,4,0,4,29,1,0,105,262,4,1
3,C*06:55-273,39,244,4,0,4,29,1,0,98,261,4,1
4,C*06:02-366,46,245,4,0,4,29,1,0,98,261,4,1
5,C*06:108-273,46,245,4,0,4,29,1,0,98,261,4,1
6,C*06:110-298,46,245,4,0,4,29,1,0,98,261,4,1


In [15]:
head(get.bad('DQB1')[order(bad.nreads/10-cov)])

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,DQB1*03:01-261,19,131,2,0,52,217,3,1,54,240,4,2
2,DQB1*03:116-261,27,131,2,0,44,212,4,1,46,235,5,2
3,DQB1*03:94-261,26,131,2,0,41,199,3,1,43,222,4,1
4,DQB1*02:02-261,31,131,2,0,34,199,3,1,36,222,4,2
5,DQB1*02:12-261,31,131,2,0,34,199,3,1,36,222,4,2
6,DQB1*02:01-261,32,131,2,0,33,199,4,2,35,222,5,3


In [16]:
rescue <- function(to.rescue, rescue.by, BAD = bad, GOOD = good){
    explained <- unique(GOOD[t %in% rescue.by, q])
    g <- sub('\\*.+', '', to.rescue)
    BAD[t %in% to.rescue & GOOD == g & !(q %in% explained), .N]
}
a <- 'A*02:01-365'
b <- 'A*03:01-365'
rescue(b, a)

[1] 4

In [17]:
system.time(rescue(a, b))

   user  system elapsed 
  0.148   0.000   0.146 

In [18]:
scoreA <- get.bad('A')
scoreA <- scoreA[cov/max(cov) > 0.95]
nrow(scoreA)

[1] 45

In [19]:
goodA <- good[t %in% scoreA$t]
badA <- bad[gene == 'A' & grepl('^A', GOOD) & q %in% goodA$q]
cand <- do.call(rbind, lapply(combn(scoreA$t, 2, simplify = F), function(p){
    h1 <- rescue(p[1], p[2], badA, goodA)
    h2 <- rescue(p[2], p[1], badA, goodA)   
    data.table(t1 = p[1], t2 = p[2], h1 = h1, h2 = h2, hopeless = h1 + h2)
}))
cand <- cand[order(hopeless)]

In [20]:
head(scoreA)

,t,bad.nreads,bad.cov,bad.nseg,bad.nbreaks,nreads2,cov2,nseg2,nbreaks2,nreads,cov,nseg,nbreaks
1,A*02:01-365,91,254,5,0,54,247,3,0,107,297,2,0
2,A*02:24-365,104,281,5,0,44,247,3,0,97,297,2,0
3,A*02:249-365,105,261,4,0,45,223,3,0,93,296,3,0
4,A*03:01-365,50,286,3,0,34,153,1,0,147,296,3,0
5,A*03:89-365,50,279,4,0,30,144,1,0,148,296,3,0
6,A*02:523-340,108,294,3,0,46,246,4,0,90,294,3,0


In [21]:
setkey(scoreA, t)
setkey(cand, t1)
cand <- cbind(cand, scoreA[cand, .(nreads, cov)])
setnames(cand, c('nreads', 'cov'), c('nreads1', 'cov1'))
setkey(cand, t2)
cand <- cbind(cand, scoreA[cand, .(nreads, cov)])
setnames(cand, c('nreads', 'cov'), c('nreads2', 'cov2'))

In [22]:
tt <- cand[, score := nreads1 + nreads2 - 10 * hopeless]
head(cand[order(-score)])

,t1,t2,h1,h2,hopeless,nreads1,cov1,nreads2,cov2,score
1,A*02:01-365,A*03:01-365,2,0,2,107,297,147,296,234
2,A*02:01-365,A*03:26-365,2,0,2,107,297,146,294,233
3,A*03:01-365,A*02:134-337,0,2,2,147,296,104,293,231
4,A*03:26-365,A*02:134-337,0,2,2,146,294,104,293,230
5,A*02:01-365,A*03:134-365,2,0,2,107,297,138,286,225
6,A*03:01-365,A*02:30-365,0,2,2,147,296,97,293,224


In [23]:
pairs <- function(G){
    scoreA <- get.bad(G)
    scoreA <- scoreA[cov/max(cov) > 0.95]
    nrow(scoreA)
    goodA <- good[t %in% scoreA$t]
    badA <- bad[gene == G & grepl(sprintf('^%s', G), GOOD) & q %in% goodA$q]
    cand <- do.call(rbind, lapply(combn(scoreA$t, 2, simplify = F), function(p){
        h1 <- rescue(p[1], p[2], badA, goodA)
        h2 <- rescue(p[2], p[1], badA, goodA)   
        data.table(t1 = p[1], t2 = p[2], h1 = h1, h2 = h2, hopeless = h1 + h2)
    }))
    cand <- cand[order(hopeless)]
    setkey(scoreA, t)
    setkey(cand, t1)
    cand <- cbind(cand, scoreA[cand, .(nreads, cov)])
    setnames(cand, c('nreads', 'cov'), c('nreads1', 'cov1'))
    setkey(cand, t2)
    cand <- cbind(cand, scoreA[cand, .(nreads, cov)])
    setnames(cand, c('nreads', 'cov'), c('nreads2', 'cov2'))
    tt <- cand[, score := nreads1 + nreads2 - 10 * hopeless]
    return(cand[order(-score)])
}

In [24]:
pairB <- pairs('B')

In [25]:
head(pairB)

,t1,t2,h1,h2,hopeless,nreads1,cov1,nreads2,cov2,score
1,B*51:01-362,B*51:193-362,0,0,0,80,297,80,297,160
2,B*51:01-362,B*51:96-337,0,0,0,80,297,79,295,159
3,B*51:193-362,B*51:96-337,0,0,0,80,297,79,295,159
4,B*51:01-362,B*51:187-362,0,0,0,80,297,78,294,158
5,B*51:193-362,B*51:187-362,0,0,0,80,297,78,294,158
6,B*51:01-362,B*51:188-362,0,0,0,80,297,78,289,158
